In [4]:
import numpy as np
import pygame

np.random.seed(42)

# Constants
Nx, Ny = 50, 50
dx=min(500/Nx, 500/Ny)
du, dv = 0.03, 0.08
au, av, bu, bv = 0.08, 0.045, -0.6, 0.0
cu, cv, Du, Dv = 0.03, -0.015, 0.02, 1.5
Fmax, Gmax = 0.2, 0.5
dt = 0.1

bv = bv-dv
au = au-du

alpha = 0

# Initialize grid
u = np.random.rand(Nx * Ny)
v = np.random.rand(Nx * Ny)

# Neighbor index calculation with periodic boundary conditions
def get_neighbors(Nx, Ny):
    x_menos = np.zeros(Nx * Ny, dtype=int)
    x_mas = np.zeros(Nx * Ny, dtype=int)
    y_menos = np.zeros(Nx * Ny, dtype=int)
    y_mas = np.zeros(Nx * Ny, dtype=int)

    for i in range(Nx):
        for j in range(Ny):
            n = i + j * Nx
            x_menos[n] = ((i - 1) % Nx) + j * Nx
            x_mas[n] = ((i + 1) % Nx) + j * Nx
            y_menos[n] = i + ((j - 1) % Ny) * Nx
            y_mas[n] = i + ((j + 1) % Ny) * Nx

    return x_menos, x_mas, y_menos, y_mas

x_menos, x_mas, y_menos, y_mas = get_neighbors(Nx, Ny)
#print(y_menos)

# Evolution step
def evolucion(u, v):
    aux_u = u.copy()
    aux_v = v.copy()

    for n in range(Nx * Ny):
        
        F=max(-Fmax, min(au * u[n] + bu * v[n] + cu, Fmax))
        G=max(-Gmax, min(av * u[n] + bv * v[n] + cv, Gmax))
        
        aux_u[n] += dt * (F + Du * (u[x_menos[n]] + u[x_mas[n]] + u[y_menos[n]] + u[y_mas[n]] - 4 * u[n]))
        aux_v[n] += dt * (G + Dv * (v[x_menos[n]] + v[x_mas[n]] + v[y_menos[n]] + v[y_mas[n]] - 4 * v[n]))

    u[:] = np.clip(aux_u, 0, 1)
    v[:] = np.clip(aux_v, 0, 1)

# Pygame visualization function
def visualize(u, t):
    screen.fill((0, 0, 0))

    # Draw grid
    minimo_u=np.min(u)
    maximo_u=np.max(u)
    for i in range(Nx):
        for j in range(Ny):
            n = i + j * Nx
            if(maximo_u==minimo_u):
                u_norm=u[n]
            else:
                u_norm=(u[n]-minimo_u)/(maximo_u-minimo_u)
            color = (int(u_norm * 255), int(u_norm * 255), int(u_norm * 255))
            pygame.draw.rect(screen, color, (i * dx, j * dx, dx, dx))

    # Render time text
    time_text = font.render(f"Time: {t * dt:.2f}", True, (255, 255, 255))
    screen.blit(time_text, (10, Ny * dx + 5))

# Pygame setup
pygame.init()
screen = pygame.display.set_mode((Nx * dx, Ny * dx + 30))  # Extra space for time display
pygame.display.set_caption("Simulation Visualization")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 24)  # Default font, size 24
# Main loop
running = True
t = 0
visualize(u, t)
while running and t < 10000:
    t += 1
    evolucion(u, v)

    if t % 10 == 0:  # Visualize every 10 steps
        visualize(u, t)
        #print(t)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    pygame.display.flip()
    clock.tick(60)  # Limit to 60 FPS

pygame.image.save(screen, "pattern_cuadrado.png")
print("Saved final state as pattern_cuadrado.png")
        
pygame.quit()

<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


pygame 2.6.1 (SDL 2.32.54, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
Saved final state as pattern_cuadrado.png


In [7]:
from PIL import Image

def save_image(u, filename="pattern_cuadrado.png"):
    # Reshape u into a 2D array
    u_2d = u.reshape((Nx, Ny))

    # Normalize values to 0-255 for grayscale image
    u_norm = (255 * (u_2d - np.min(u_2d)) / (np.max(u_2d) - np.min(u_2d))).astype(np.uint8)

    # Save as an image
    img = Image.fromarray(u_norm, mode="L")
    img.save(filename)
    
save_image(u)

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
from PIL import Image

# Load image as grayscale
img = Image.open("result.png").convert("L")
data = np.array(img)
Ny, Nx = data.shape

# Normalize to [0, 1]
data_norm = data.astype(float) / 255.0

# Generate square patches
patches = []
colors = []

for y in range(Ny):
    for x in range(Nx):
        val = data_norm[y, x]
        gray = (val, val, val)
        rect = Rectangle((x, Ny - y - 1), 1, 1)
        patches.append(rect)
        colors.append(gray)

# Plot with PatchCollection
fig, ax = plt.subplots(figsize=(10, 10), dpi=300)
collection = PatchCollection(patches, facecolors=colors, edgecolors=colors, linewidths=0)
ax.add_collection(collection)

ax.set_xlim(0, Nx)
ax.set_ylim(0, Ny)
ax.set_aspect('equal')
ax.axis('off')

plt.tight_layout()
plt.savefig("square_grid_visualization.png", dpi=600, bbox_inches='tight')
plt.close()

print("✅ Image saved as square_grid_visualization.png")


✅ Image saved as square_grid_visualization.png
